In [8]:
import os
import pandas as pd
import numpy as np
import time
from tqdm import tqdm

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim 


import warnings
from torch.utils.data import Dataset, DataLoader

warnings.filterwarnings('ignore')


In [3]:
cur_path = os.getcwd()
data_path = os.path.join(cur_path, 'data')

train_click_log = pd.read_csv(os.path.join(data_path, 'train_click_log.csv'))

articles_emb = pd.read_csv(os.path.join(data_path, 'articles_emb.csv'))

test_click_log = pd.read_csv(os.path.join(data_path, 'testA_click_log.csv'))

articles  = pd.read_csv(os.path.join(data_path, 'articles.csv'))

In [4]:
train_click_log.head()

,user_id,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
0,199999,160417,1507029570190,4,1,17,1,13,1
1,199999,5408,1507029571478,4,1,17,1,13,1
2,199999,50823,1507029601478,4,1,17,1,13,1
3,199998,157770,1507029532200,4,1,17,1,25,5
4,199998,96613,1507029671831,4,1,17,1,25,5


In [5]:
articles.head()

,article_id,category_id,created_at_ts,words_count
0,0,0,1513144419000,168
1,1,1,1405341936000,189
2,2,1,1408667706000,250
3,3,1,1408468313000,230
4,4,1,1407071171000,162


In [6]:
articles_emb.head()

,article_id,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,...,emb_240,emb_241,emb_242,emb_243,emb_244,emb_245,emb_246,emb_247,emb_248,emb_249
0,0,-0.161183,-0.957233,-0.137944,0.050855,0.830055,0.901365,-0.335148,-0.559561,-0.500603,...,0.321248,0.313999,0.636412,0.169179,0.540524,-0.813182,0.286870,-0.231686,0.597416,0.409623
1,1,-0.523216,-0.974058,0.738608,0.155234,0.626294,0.485297,-0.715657,-0.897996,-0.359747,...,-0.487843,0.823124,0.412688,-0.338654,0.320786,0.588643,-0.594137,0.182828,0.397090,-0.834364
2,2,-0.619619,-0.972960,-0.207360,-0.128861,0.044748,-0.387535,-0.730477,-0.066126,-0.754899,...,0.454756,0.473184,0.377866,-0.863887,-0.383365,0.137721,-0.810877,-0.447580,0.805932,-0.285284
3,3,-0.740843,-0.975749,0.391698,0.641738,-0.268645,0.191745,-0.825593,-0.710591,-0.040099,...,0.271535,0.036040,0.480029,-0.763173,0.022627,0.565165,-0.910286,-0.537838,0.243541,-0.885329
4,4,-0.279052,-0.972315,0.685374,0.113056,0.238315,0.271913,-0.568816,0.341194,-0.600554,...,0.238286,0.809268,0.427521,-0.615932,-0.503697,0.614450,-0.917760,-0.424061,0.185484,-0.580292


### 1.节省内存函数

In [ ]:
def reduce_memory(df):
    start_time = time.time()

    numeric_cols = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

    start_memory = df.memory_usage().sum() / 1024 ** 2

    for col in df.columns:
        col_type = df[col].dtype

        if col_type in numeric_cols:
            c_min = df[col].min()
            c_max = df[col].max()

            if pd.isnull(c_min) or pd.isnull(c_max):
                continue

            if str(col_type) == 'int':    
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
            


-- Mem. usage decreased to 23.34 Mb (69.4% reduction),time spend:0.00 min


,user_id,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
0,199999,160417,1507029570190,4,1,17,1,13,1
1,199999,5408,1507029571478,4,1,17,1,13,1
2,199999,50823,1507029601478,4,1,17,1,13,1
3,199998,157770,1507029532200,4,1,17,1,25,5
4,199998,96613,1507029671831,4,1,17,1,25,5
...,...,...,...,...,...,...,...,...,...
1112618,33446,16346,1508212054157,4,3,2,1,25,1
1112619,0,30760,1508211672520,4,1,17,1,25,2
1112620,0,157507,1508211702520,4,1,17,1,25,2
1112621,199178,234481,1508211513583,4,3,2,1,25,2
